# Spin ice Debye-Hueckel model

In [1]:
###Imports###
%matplotlib notebook

import numpy as np
import mpmath as mp
mp.dps = 15; mp.pretty = True
from scipy.integrate import cumtrapz
import matplotlib.pyplot as plt
import random

In [2]:
###Global constants###
N_0 = 6.02214 * 10**23 #Total number of site, avagadro's number
k_B = 1.38064852 * 10**-23 #Boltzmann constant

##Dysprosium titanate##


##water ice##

In [3]:
###Global variables###
factor = 4 #coupling factor
factor_string = '4'
meta_switch = True #switch for turning on (True) or off (False) the metastable state (corrected from older version)
hysteresis_switch = True #switch to enable (True) or disable (False) hysteresis

In [4]:
def double_monopole(start_mu, coupling, switch, hysteresis,  start_T, num_T, T_max, tol, limit, n_init=0, n_init2=1):

    mu_0 = 4 * np.pi * 10**-7 #vacuum permeability
    a = 4.34 * 10**-10 #lattice spacing of diamond lattice
    Q = 4.28 * 10**-13 #magnetic charge
    vol_per_site = ((10.1**3)/8) * 10**-30
    N_0 = 6.02214 * 10**23 #Total number of site, avagadro's number
    k_B = 1.38064852 * 10**-23 #Boltzmann constant
    T = start_T #Temperature in Kelvins (parameter)
    
    ##array initialisation##
    #TempArray_double = []
    TempArray_double = np.geomspace(start_T, T_max, num=num_T)
    SpecHeatArray_double = []
    EntropyArray = []
    muDoubleArray = []
    mu2DoubleArray = []
    nArray = []
    n_2Array = []
    internalEnergyArray = []
    
    ##hysteresis modification##
    if hysteresis == True:
        #appends a reversed TempArray onto the original TempArray
        TempArray_double = np.append(TempArray_double, np.flip(TempArray_double), axis=0)
        num_T = num_T * 2 # correcting the number of T points to match TempArray number of elements
    
    j=0 #variable to cycle through elements of TempArray
    
    old_mu = 0
    new_mu = -start_mu * k_B
    init_mu = -start_mu * k_B
    old_mu_2 = 0
    new_mu_2 = -start_mu * coupling * k_B
    init_mu_2 = -start_mu * coupling * k_B
    
    #while T <= T_max :
    while j < num_T :
        
        i = 0
        old_mu = 0
        old_mu_2 = 0
        #note older versions had lines here that reset the new_mu values
        #this would stop previous temperature steps affecting subsequent ones
        #thus stopping any hysteresis
        beta = 1/(k_B * T)
        l_T = (mu_0 * Q * Q)/(k_B * T * 8 * np.pi)
        #print("Temperature:", T)
        
        
        while np.abs((new_mu - old_mu)/new_mu) > tol and i < limit :
            #print("new mu:", new_mu)
            old_mu = new_mu #if new and old values not close enough, new replaces old
            old_mu_2 = new_mu_2
            #new number of monopoles
            n = (4/3 * np.exp(beta * old_mu))/(1 +(1/3 * ( 4 * np.exp(beta * old_mu) + np.exp(beta * old_mu_2))))
            n_2 = (1/3 * np.exp(beta * old_mu_2))/(1 +(1/3 * ( 4 * np.exp(beta * old_mu) + np.exp(beta * old_mu_2))))
            #print("n:", n)
            
            #calculating new charge densities
            ##this 'if' statement switches on (switch=True) and off (switch=False) metastability
            ###As such hysteresis will not occur if this switch=False
            if i == 0 and switch == False:
                #print("metastability on")
                rho = n_init/vol_per_site
                rho_2 = n_init2/vol_per_site
                
            else:
                rho = n/vol_per_site
                rho_2 = n_2/vol_per_site
                
            rho_I = rho + 4*rho_2
            #print("rho:", rho)
            
            l_D = np.sqrt((k_B * T)/(Q * Q * rho_I * mu_0)) #Debye length Calc
            #print("l_D: ", l_D)
            #print("l_T:", l_T)
            
            deltaDH = k_B * T * ((l_T)/(l_D + a))
            new_mu = init_mu + deltaDH
            new_mu_2 = init_mu_2 + 4*deltaDH
            #print("deltaDH:",deltaDH)
            #print("mu:", new_mu)
            #print("mu_2:", new_mu_2)
            #print(old_mu)
            #print(np.abs(new_mu - old_mu))
            #print("")
            
            i += 1
            if i==limit:
                print('Warning: Monopole density did not converge after', limit, 'iterations!')
                # This stops the function running forever if there is a problem with convergence.
        
        ##Calculates the differential of monopole density n wit respect to Temperature, used to calculate specific heat
        #differential = mp.diff(lambda t:((4/3 * mp.exp((1/(k_B * t)) * old_mu))/(1 +(1/3 * ( 4 * mp.exp((1/(k_B * t)) * old_mu) + mp.exp((1/(k_B * t)) * old_mu_2) )))), T)
        #differential_2 = mp.diff(lambda t:((1/3 * mp.exp((1/(k_B * t)) * old_mu_2))/(1 +(1/3 * ( 4 * mp.exp((1/(k_B * t)) * old_mu) + mp.exp((1/(k_B * t)) * old_mu_2) )))), T)
        
        #dn/dT calculations, long because of the equations being self-consistent
        differential = mp.diff(lambda t:((1/3 * 4 * mp.exp((1/(k_B * t)) * (init_mu + k_B * t * (((mu_0 * Q * Q)/(k_B * t * 8 * mp.pi))/((mp.sqrt((k_B * t)/(Q * Q * rho_I * mu_0))) + a))) ))/(1 +(1/3 * ( (4 * mp.exp((1/(k_B * t)) * (init_mu + k_B * t * (((mu_0 * Q * Q)/(k_B * t * 8 * mp.pi))/((mp.sqrt((k_B * t)/(Q * Q * rho_I * mu_0))) + a))))) + mp.exp((1/(k_B * t)) * (init_mu_2 + coupling * k_B * t * (((mu_0 * Q * Q)/(k_B * t * 8 * mp.pi))/((mp.sqrt((k_B * t)/(Q * Q * rho_I * mu_0))) + a)))) ) ))), T)
        differential_2 = mp.diff(lambda t:((1/3 * mp.exp((1/(k_B * t)) * (init_mu_2 + coupling * k_B * t * (((mu_0 * Q * Q)/(k_B * t * 8 * mp.pi))/((mp.sqrt((k_B * t)/(Q * Q * rho_I * mu_0))) + a)))))/(1 +(1/3 * ( 4 * mp.exp((1/(k_B * t)) * (init_mu + k_B * t * (((mu_0 * Q * Q)/(k_B * t * 8 * mp.pi))/((mp.sqrt((k_B * t)/(Q * Q * rho_I * mu_0))) + a)))) + mp.exp((1/(k_B * t)) * (init_mu_2 + coupling * k_B * t * (((mu_0 * Q * Q)/(k_B * t * 8 * mp.pi))/((mp.sqrt((k_B * t)/(Q * Q * rho_I * mu_0))) + a)))) )))), T)
        
        SpecHeatVal = - N_0 * (new_mu * float(differential) + new_mu_2 * float(differential_2))
        #print("differential of n wrt T:", differential)
        #print("differential of n_2 wrt T:", differential_2)
        #print("Specific Heat Value:", SpecHeatVal)
        
        Entropy = - k_B * N_0 *( (n*np.log(n/2) + (n_2*np.log(2*n_2) + ((1-n-n_2)*np.log(1-n-n_2)) + ((1-n-n_2)*np.log(2/3)) ) ) )
        #Entropy_corrected = Entropy - (8.314 * 1/2*np.log(3/2))
        
        if Entropy < 0:
            Entropy = 0
        
        internalEnergyTerm = np.log(1+(a/l_D)) - (a/l_D) + 1/2*((a/l_D)**2)
        #internalEnergyTerm = np.log(1+(a/np.sqrt((k_B*T)/(Q*Q*n/vol_per_site*mu_0)))) - (a/np.sqrt((k_B*T)/(Q * Q * n/vol_per_site * mu_0))) + 1/2*((a/np.sqrt((k_B * T)/(Q * Q * n/vol_per_site * mu_0)))**2)
        internalEnergy = - ((4 * N_0 * k_B * T)/(6*np.pi*np.sqrt(3))) * internalEnergyTerm
        
        nArray = np.append(nArray, n)
        n_2Array = np.append(n_2Array, n_2)
        muDoubleArray = np.append(muDoubleArray, new_mu)
        mu2DoubleArray = np.append(mu2DoubleArray, new_mu_2)
        EntropyArray = np.append(EntropyArray, Entropy)
        internalEnergyArray = np.append(internalEnergyArray, internalEnergy)
        #TempArray_double = np.append(TempArray_double, T)
        SpecHeatArray_double = np.append(SpecHeatArray_double, SpecHeatVal)
        
        #T += T_step
        
        j += 1
        if j < num_T :
            T = TempArray_double[j]
        #print("done")
        
    Omega = internalEnergyArray - (init_mu * (N_0*nArray)) - (init_mu_2 * (N_0*n_2Array)) - (TempArray_double*EntropyArray)
        
    print("")
    print("final")
    #print("Temperature values:", TempArray_double)
    #print("Specific Heat array:", SpecHeatArray_double)
    return (TempArray_double, SpecHeatArray_double, EntropyArray, muDoubleArray, mu2DoubleArray, nArray, n_2Array, EntropyArray, Omega)

In [5]:
###Quickrun###

###1.00 mu_1 values, double monopole###
TempArray_double1, SpecHeatArray_double1, EntropyArray_double1, muDoubleArray1, mu2DoubleArray1, nArray_double1, n_2Array_double1, EntropyArray_double1, Omega_double1 = double_monopole(1.30, factor, False, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)


C:\Users\Omar3\Anaconda3\lib\site-packages\ipykernel_launcher.py:107: RuntimeWarning: divide by zero encountered in log
C:\Users\Omar3\Anaconda3\lib\site-packages\ipykernel_launcher.py:107: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Omar3\Anaconda3\lib\site-packages\ipykernel_launcher.py:107: RuntimeWarning: invalid value encountered in log



final


In [ ]:
###0.50 mu_1 values, double monopole###
TempArray_double1, SpecHeatArray_double1, EntropyArray_double1, muDoubleArray1, mu2DoubleArray1, nArray_double1, n_2Array_double1, EntropyArray_double1, Omega_double1 = double_monopole(0.50, factor, meta_switch, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)

###0.75 mu_1 values, double monopole###
TempArray_double9, SpecHeatArray_double9, EntropyArray_double9, muDoubleArray9, mu2DoubleArray9, nArray_double9, n_2Array_double9, EntropyArray_double9, Omega_double9 = double_monopole(0.75, factor, meta_switch, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)

###1.00 mu_1 values, double monopole###
TempArray_double2, SpecHeatArray_double2, EntropyArray_double2, muDoubleArray2, mu2DoubleArray2, nArray_double2, n_2Array_double2, EntropyArray_double2, Omega_double2 = double_monopole(1.00, factor, meta_switch, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)

###1.15 mu_1 values, double monopole###
TempArray_double3, SpecHeatArray_double3, EntropyArray_double3, muDoubleArray3, mu2DoubleArray3, nArray_double3, n_2Array_double3, EntropyArray_double3, Omega_double3 = double_monopole(1.15, factor, meta_switch, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)

###1.30 mu_1 values, double monopole###7
TempArray_double4, SpecHeatArray_double4, EntropyArray_double4, muDoubleArray4, mu2DoubleArray4, nArray_double4, n_2Array_double4, EntropyArray_double4, Omega_double4 = double_monopole(1.30, factor, meta_switch, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)

###1.57 mu_1 values, double monopole###
TempArray_double5, SpecHeatArray_double5, EntropyArray_double5, muDoubleArray5, mu2DoubleArray5, nArray_double5, n_2Array_double5, EntropyArray_double5, Omega_double5 = double_monopole(1.57, factor, meta_switch, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)

###1.90 mu_1 values, double monopole###
TempArray_double6, SpecHeatArray_double6, EntropyArray_double6, muDoubleArray6, mu2DoubleArray6, nArray_double6, n_2Array_double6, EntropyArray_double6, Omega_double6 = double_monopole(1.90, factor, meta_switch, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)

###2.50 mu_1 values, double monopole###
TempArray_double7, SpecHeatArray_double7, EntropyArray_double7, muDoubleArray7, mu2DoubleArray7, nArray_double7, n_2Array_double7, EntropyArray_double7, Omega_double7 = double_monopole(2.50, factor, meta_switch, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)

###4.35 mu_1 values, double monopole###
TempArray_double8, SpecHeatArray_double8, EntropyArray_double8, muDoubleArray8, mu2DoubleArray8, nArray_double8, n_2Array_double8, EntropyArray_double8, Omega_double8 = double_monopole(4.35, factor, meta_switch, hysteresis_switch, 0.01, 1000, 50, 1e-4, 1000)
